In [17]:
import argparse,os,logging,psutil,time
from joblib import Parallel,delayed

#import utils file
%run utils.ipynb
%run make_graph2vec_corpus.ipynb
%run train_utils.ipynb
%run classify_combined.ipynb

logger = logging.getLogger()
logger.setLevel("INFO")

In [2]:
args = argparse.ArgumentParser("graph2vec")
    
args.add_argument("-c","--corpus", default = "../data/kdd_datasets/ptc",
                  help="Path to directory containing graph files to be used for graph classification or clustering")

args.add_argument('-l','--class_labels_file_name', default='../data/kdd_datasets/ptc.Labels',
                  help='File name containg the name of the sample and the class labels')

args.add_argument('-o', "--output_dir", default = "../embeddings",
                  help="Path to directory for storing output embeddings")

args.add_argument('-b',"--batch_size", default=128, type=int,
                  help="Number of samples per training batch")

args.add_argument('-e',"--epochs", default=1000, type=int,
                  help="Number of iterations the whole dataset of graphs is traversed")

args.add_argument('-d',"--embedding_size", default=1024, type=int,
                  help="Intended graph embedding size to be learnt")

args.add_argument('-neg', "--num_negsample", default=10, type=int,
                  help="Number of negative samples to be used for training")

args.add_argument('-lr', "--learning_rate", default=0.3, type=float,
                  help="Learning rate to optimize the loss function")

args.add_argument("--wlk_h", default=3, type=int, help="Height of WL kernel (i.e., degree of rooted subgraph "
                                                       "features to be considered for representation learning)")

args.add_argument('-lf', '--label_filed_name', default='Label', help='Label field to be used '
                                                                     'for coloring nodes in graphs using WL kenrel')

#args.parse_args()

_StoreAction(option_strings=['-lf', '--label_filed_name'], dest='label_filed_name', nargs=None, const=None, default='Label', type=None, choices=None, help='Label field to be used for coloring nodes in graphs using WL kenrel', metavar=None)

In [3]:
# Set it manually for now
corpus_dir = "/home/ipsita/BTP/graph2vec/data/kdd_datasets/mutag"
output_dir = "/home/ipsita/BTP/graph2vec/embeddings"
batch_size = 128
epochs = 100
embedding_size = 1024
num_negsample = 10
learning_rate = 0.3
wlk_h = 3
label_filed_name = 'Label'
class_labels_fname = '/home/ipsita/BTP/graph2vec/data/kdd_datasets/mutag.Labels'

wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

In [8]:
graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))

INFO:root:Loaded 344 graph file names form /home/ipsita/BTP/graph2vec/data/kdd_datasets/ptc


In [9]:
t0 = time()
wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
logging.info('dumped sg2vec sentences in {} sec.'.format(time() - t0))

loaded all graphs in 0.33 sec
initial relabeling done in 0.06 sec
WL iteration 1 done in 0.06 sec.
num of WL rooted subgraphs in iter 1 is 160
WL iteration 2 done in 0.07 sec.
num of WL rooted subgraphs in iter 2 is 1038


INFO:root:dumped sg2vec sentences in 0.6182849407196045 sec.


WL iteration 3 done in 0.09 sec.
num of WL rooted subgraphs in iter 3 is 2624
dumped sg2vec sentences in 0.0


In [22]:
t0 = time()
embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                 epochs, batch_size, output_dir)
logging.info('Trained the skipgram model in {} sec.'.format(round(time()-t0, 2)))

INFO:root:Initializing SKIPGRAM...
INFO:root:number of graphs: 344
INFO:root:subgraph vocabulary size: 3804
INFO:root:total number of subgraphs to be trained: 34837
INFO:root:Epoch: 0 : Average loss for step: 100 : 44.053448
INFO:root:Epoch: 0 : Average loss for step: 200 : 37.702990
INFO:root:#########################   Epoch: 0 :  35.445183, 1.09 sec.  #####################
INFO:root:Epoch: 1 : Average loss for step: 100 : 25.386242
INFO:root:Epoch: 1 : Average loss for step: 200 : 24.069743
INFO:root:#########################   Epoch: 1 :  22.902555, 0.97 sec.  #####################
INFO:root:Epoch: 2 : Average loss for step: 100 : 15.687668
INFO:root:Epoch: 2 : Average loss for step: 200 : 16.182734
INFO:root:#########################   Epoch: 2 :  15.780944, 0.97 sec.  #####################
INFO:root:Epoch: 3 : Average loss for step: 100 : 13.728236
INFO:root:Epoch: 3 : Average loss for step: 200 : 12.342629
INFO:root:#########################   Epoch: 3 :  12.033786, 0.96 sec.  #

INFO:root:Epoch: 38 : Average loss for step: 100 : 1.192453
INFO:root:Epoch: 38 : Average loss for step: 200 : 1.208227
INFO:root:#########################   Epoch: 38 :  1.186123, 0.95 sec.  #####################
INFO:root:Epoch: 39 : Average loss for step: 100 : 1.231534
INFO:root:Epoch: 39 : Average loss for step: 200 : 1.195807
INFO:root:#########################   Epoch: 39 :  1.178413, 0.94 sec.  #####################
INFO:root:Epoch: 40 : Average loss for step: 100 : 1.222653
INFO:root:Epoch: 40 : Average loss for step: 200 : 1.195820
INFO:root:#########################   Epoch: 40 :  1.207544, 0.95 sec.  #####################
INFO:root:Epoch: 41 : Average loss for step: 100 : 1.234114
INFO:root:Epoch: 41 : Average loss for step: 200 : 1.215157
INFO:root:#########################   Epoch: 41 :  1.199390, 0.94 sec.  #####################
INFO:root:Epoch: 42 : Average loss for step: 100 : 1.153730
INFO:root:Epoch: 42 : Average loss for step: 200 : 1.154644
INFO:root:##############

INFO:root:#########################   Epoch: 76 :  0.919826, 0.94 sec.  #####################
INFO:root:Epoch: 77 : Average loss for step: 100 : 0.904998
INFO:root:Epoch: 77 : Average loss for step: 200 : 0.923373
INFO:root:#########################   Epoch: 77 :  0.934361, 0.93 sec.  #####################
INFO:root:Epoch: 78 : Average loss for step: 100 : 0.918342
INFO:root:Epoch: 78 : Average loss for step: 200 : 0.922286
INFO:root:#########################   Epoch: 78 :  0.915841, 0.93 sec.  #####################
INFO:root:Epoch: 79 : Average loss for step: 100 : 0.942286
INFO:root:Epoch: 79 : Average loss for step: 200 : 0.933504
INFO:root:#########################   Epoch: 79 :  0.910514, 0.93 sec.  #####################
INFO:root:Epoch: 80 : Average loss for step: 100 : 0.955701
INFO:root:Epoch: 80 : Average loss for step: 200 : 0.930187
INFO:root:#########################   Epoch: 80 :  0.925473, 0.94 sec.  #####################
INFO:root:Epoch: 81 : Average loss for step: 100 :

In [24]:
perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)

INFO:root:Y (label) matrix shape: (344,)
INFO:root:Train and Test matrix shapes: (309, 1024), (35, 1024), (309,), (35,) 


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    8.8s finished
INFO:root:best classifier model's hyperparamters
INFO:root:Linear SVM accuracy: 0.4857142857142857
INFO:root:             precision    recall  f1-score   support

         -1       0.40      0.57      0.47        14
          1       0.60      0.43      0.50        21

avg / total       0.52      0.49      0.49        35



# To run many time

In [35]:
# These parts need not be repeated

####################### set parametrs #######################################
corpus_dir = "/home/ipsita/BTP/graph2vec/data/kdd_datasets/mutag"
output_dir = "/home/ipsita/BTP/graph2vec/embeddings"
batch_size = 128
epochs = 1000
embedding_size = 1024
num_negsample = 10
learning_rate = 0.3
wlk_h = 3
label_filed_name = 'Label'
class_labels_fname = '/home/ipsita/BTP/graph2vec/data/kdd_datasets/mutag.Labels'

wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

######################## load graphs from corpus ###############################
graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))

######################## make corpus files #####################################
t0 = time.time()
wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
logging.info('dumped sg2vec sentences in {} sec.'.format(time.time() - t0))

INFO:root:Loaded 188 graph file names form /home/ipsita/BTP/graph2vec/data/kdd_datasets/mutag
INFO:root:dumped sg2vec sentences in 0.7712142467498779 sec.


loaded all graphs in 0.65 sec
initial relabeling done in 0.03 sec
WL iteration 1 done in 0.03 sec.
num of WL rooted subgraphs in iter 1 is 33
WL iteration 2 done in 0.03 sec.
num of WL rooted subgraphs in iter 2 is 174
WL iteration 3 done in 0.04 sec.
num of WL rooted subgraphs in iter 3 is 572
dumped sg2vec sentences in 0.0


In [36]:
total_acc = []
fname = "/home/ipsita/BTP/graph2vec/embeddings/mutag_dims_1024_epochs_1000_lr_0.3_embeddings.txt"

for i in range(0, 1):
    
    if os.path.isfile(fname):
        os.remove(fname);

    ######################### training ##############################################
    t0 = time.time()
    embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                     epochs, batch_size, output_dir)
    print(embedding_fname)
    logging.info('Trained the skipgram model in {} sec.'.format(round(time.time()-t0, 2)))
    
    ######################## classify ###############################################
    a = perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)
    print('i=',i,'accuracy=',a)
    total_acc.append(a)
    

INFO:root:Initializing SKIPGRAM...
INFO:root:number of graphs: 188
INFO:root:subgraph vocabulary size: 787
INFO:root:total number of subgraphs to be trained: 13485
INFO:root:#########################   Epoch: 0 :  19.721200, 0.91 sec.  #####################
INFO:root:#########################   Epoch: 1 :  10.679841, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 2 :  7.999816, 0.65 sec.  #####################
INFO:root:#########################   Epoch: 3 :  5.720646, 0.71 sec.  #####################
INFO:root:#########################   Epoch: 4 :  4.270865, 0.82 sec.  #####################
INFO:root:#########################   Epoch: 5 :  4.251593, 0.72 sec.  #####################
INFO:root:#########################   Epoch: 6 :  3.138256, 0.94 sec.  #####################
INFO:root:#########################   Epoch: 7 :  2.626400, 0.81 sec.  #####################
INFO:root:#########################   Epoch: 8 :  2.166890, 0.87 sec.  ###################

INFO:root:#########################   Epoch: 86 :  0.851317, 0.60 sec.  #####################
INFO:root:#########################   Epoch: 87 :  0.836060, 0.60 sec.  #####################
INFO:root:#########################   Epoch: 88 :  0.838333, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 89 :  0.855861, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 90 :  0.832031, 0.73 sec.  #####################
INFO:root:#########################   Epoch: 91 :  0.844279, 0.65 sec.  #####################
INFO:root:#########################   Epoch: 92 :  0.872640, 0.78 sec.  #####################
INFO:root:#########################   Epoch: 93 :  0.859076, 0.66 sec.  #####################
INFO:root:#########################   Epoch: 94 :  0.835596, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 95 :  0.833519, 0.68 sec.  #####################
INFO:root:#########################   Epoch: 96 :  0.845355,

INFO:root:#########################   Epoch: 173 :  0.769886, 0.89 sec.  #####################
INFO:root:#########################   Epoch: 174 :  0.783928, 0.99 sec.  #####################
INFO:root:#########################   Epoch: 175 :  0.762848, 0.81 sec.  #####################
INFO:root:#########################   Epoch: 176 :  0.766613, 0.91 sec.  #####################
INFO:root:#########################   Epoch: 177 :  0.793124, 0.93 sec.  #####################
INFO:root:#########################   Epoch: 178 :  0.748674, 0.99 sec.  #####################
INFO:root:#########################   Epoch: 179 :  0.760866, 0.96 sec.  #####################
INFO:root:#########################   Epoch: 180 :  0.746049, 0.88 sec.  #####################
INFO:root:#########################   Epoch: 181 :  0.776825, 1.14 sec.  #####################
INFO:root:#########################   Epoch: 182 :  0.766383, 1.09 sec.  #####################
INFO:root:#########################   Epoch: 183 :

INFO:root:#########################   Epoch: 260 :  0.712455, 1.06 sec.  #####################
INFO:root:#########################   Epoch: 261 :  0.731000, 0.78 sec.  #####################
INFO:root:#########################   Epoch: 262 :  0.717781, 0.70 sec.  #####################
INFO:root:#########################   Epoch: 263 :  0.706159, 0.81 sec.  #####################
INFO:root:#########################   Epoch: 264 :  0.725730, 0.76 sec.  #####################
INFO:root:#########################   Epoch: 265 :  0.719096, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 266 :  0.707024, 0.82 sec.  #####################
INFO:root:#########################   Epoch: 267 :  0.731322, 0.74 sec.  #####################
INFO:root:#########################   Epoch: 268 :  0.724972, 0.66 sec.  #####################
INFO:root:#########################   Epoch: 269 :  0.725046, 0.71 sec.  #####################
INFO:root:#########################   Epoch: 270 :

INFO:root:#########################   Epoch: 347 :  0.696757, 0.65 sec.  #####################
INFO:root:#########################   Epoch: 348 :  0.687280, 0.67 sec.  #####################
INFO:root:#########################   Epoch: 349 :  0.704446, 0.67 sec.  #####################
INFO:root:#########################   Epoch: 350 :  0.687601, 0.71 sec.  #####################
INFO:root:#########################   Epoch: 351 :  0.699001, 0.71 sec.  #####################
INFO:root:#########################   Epoch: 352 :  0.684839, 0.67 sec.  #####################
INFO:root:#########################   Epoch: 353 :  0.711818, 0.67 sec.  #####################
INFO:root:#########################   Epoch: 354 :  0.690072, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 355 :  0.679808, 0.67 sec.  #####################
INFO:root:#########################   Epoch: 356 :  0.715876, 0.69 sec.  #####################
INFO:root:#########################   Epoch: 357 :

...

INFO:root:#########################   Epoch: 869 :  0.634729, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 870 :  0.646627, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 871 :  0.640814, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 872 :  0.642058, 0.65 sec.  #####################
INFO:root:#########################   Epoch: 873 :  0.607206, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 874 :  0.647981, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 875 :  0.641327, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 876 :  0.624279, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 877 :  0.645010, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 878 :  0.627657, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 879 :

INFO:root:#########################   Epoch: 956 :  0.645193, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 957 :  0.651190, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 958 :  0.624104, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 959 :  0.634628, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 960 :  0.630049, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 961 :  0.624937, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 962 :  0.638785, 0.64 sec.  #####################
INFO:root:#########################   Epoch: 963 :  0.629789, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 964 :  0.632291, 0.63 sec.  #####################
INFO:root:#########################   Epoch: 965 :  0.639892, 0.62 sec.  #####################
INFO:root:#########################   Epoch: 966 :

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    3.7s finished
INFO:root:best classifier model's hyperparamters
INFO:root:Linear SVM accuracy: 0.9473684210526315
INFO:root:             precision    recall  f1-score   support

         -1       1.00      0.86      0.92         7
          1       0.92      1.00      0.96        12

avg / total       0.95      0.95      0.95        19



i= 0 accuracy= 0.9473684210526315


In [ ]:
total_acc

In [18]:
########## temp cell to classify combined embeddings ################
fname = "/home/ipsita/BTP/graph2vec/embeddings/mutag_dims_1024_epochs_1000_lr_0.3_embeddings.txt"
a = perform_classification (corpus_dir, wl_extn, fname, class_labels_fname)
print('accuracy =',a)

INFO:root:Y (label) matrix shape: (188,)
INFO:root:Train and Test matrix shapes: (169, 2048), (19, 2048), (169,), (19,) 


X shape (188, 2048)
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    3.1s finished
INFO:root:best classifier model's hyperparamters
INFO:root:Linear SVM accuracy: 0.7894736842105263
INFO:root:             precision    recall  f1-score   support

         -1       0.71      0.71      0.71         7
          1       0.83      0.83      0.83        12

avg / total       0.79      0.79      0.79        19



accuracy = 0.7894736842105263


In [12]:
avg = 0
for i in range(0, 5):
    avg += total_acc[i]
avg/5

NameError: name 'total_acc' is not defined

In [24]:
import statistics

statistics.stdev(total_acc)

0.128920512778062

In [49]:
import argparse,os,logging,psutil,time
from joblib import Parallel,delayed

from utils import get_files
from train_utils import train_skipgram
from classify import perform_classification
from make_graph2vec_corpus import *
from time import time

logger = logging.getLogger()
logger.setLevel("INFO")


def main(args):
    
    '''
    :param args: arguments for
    1. training the skigram model for learning subgraph representations
    2. construct the deep WL kernel using the learnt subgraph representations
    3. performing graph classification using  the WL and deep WL kernel
    :return: None
    '''
    corpus_dir = args.corpus
    output_dir = args.output_dir
    batch_size = args.batch_size
    epochs = args.epochs
    embedding_size = args.embedding_size
    num_negsample = args.num_negsample
    learning_rate = args.learning_rate
    wlk_h = args.wlk_h
    label_filed_name = args.label_filed_name
    class_labels_fname = args.class_labels_file_name

    wl_extn = 'g2v'+str(wlk_h)    # wlk_h is height to be considered

    assert os.path.exists(corpus_dir), "File {} does not exist".format(corpus_dir)
    assert os.path.exists(output_dir), "Dir {} does not exist".format(output_dir)

    graph_files = get_files(dirname=corpus_dir, extn='.gexf', max_files=0)
    logging.info('Loaded {} graph file names form {}'.format(len(graph_files),corpus_dir))


    t0 = time()
    wlk_relabel_and_dump_memory_version(graph_files, max_h=wlk_h, node_label_attr_name=label_filed_name)
    logging.info('dumped sg2vec sentences in {} sec.'.format(time() - t0))

    t0 = time()
    embedding_fname = train_skipgram(corpus_dir, wl_extn, learning_rate, embedding_size, num_negsample,
                                     epochs, batch_size, output_dir)
    logging.info('Trained the skipgram model in {} sec.'.format(round(time()-t0, 2)))

    perform_classification (corpus_dir, wl_extn, embedding_fname, class_labels_fname)

ModuleNotFoundError: No module named 'utils'